# Introduction

I'm debugging [this issue](https://github.com/ElArkk/jax-unirep/issues/59).

To reproduce this notebook, activate an isolated Python environment, then install jax-unirep from this branch:

```
pip install git+https://github.com/ElArkk/jax-unirep.git@oscillating-59
```

In that branch, all of the logging statements have been added in.

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [17]:
from jax_unirep import evotune, fit
from jax_unirep.utils import load_params_1900
import logging

logger = logging.getLogger("NOTEBOOK")
logger.setLevel(logging.DEBUG)

In [60]:
sequences = ["MKLVIPJ", "MMLVIKJP", "MKLVIJJ"]
params = fit(params=None, sequences=sequences, n_epochs=10)

DEBUG:root:{8}


INFO:evotuning:Length-batching done: 1 unique lengths, with average length 3.0, max length 3 and min length 3.


INFO:evotuning:Starting epoch 1
DEBUG:root:Calculating average loss.


DEBUG:root:Input shape: (3, 9, 10)
DEBUG:root:Output shape: (3, 9, 25)
DEBUG:layers.py:apply_fun:id of params: 5201263984
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/1)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5201263984
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/1)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/1)>
DEBUG:absl:Compiling evotune_loss for args (ShapedArray(float32[7600]), ShapedArray(float32[7600]), ShapedArray(float32[1900]), ShapedArray(float32[1900]), ShapedArray(float32[7600]), ShapedArray(float32[1900,7600]), ShapedArray(float32[1900,1900]), ShapedArray(float32[10,1900]), ShapedArray(float32[10,7600]), ShapedArray(float32[1900,25]), ShapedArray(float32[25]), ShapedArray(float32[3,9,10]), ShapedArray(float32[3,9,25])).


INFO:evotuning:Epoch 0: Estimated average training-set loss: 0.48966851830482483. Weights dumped.


...

INFO:evotuning:Epoch 5: Estimated average training-set loss: 0.3723492920398712. Weights dumped.
INFO:evotuning:Starting epoch 7
DEBUG:root:Calculating average loss.


INFO:evotuning:Epoch 6: Estimated average training-set loss: 0.34603431820869446. Weights dumped.
INFO:evotuning:Starting epoch 8
DEBUG:root:Calculating average loss.


INFO:evotuning:Epoch 7: Estimated average training-set loss: 0.3213047683238983. Weights dumped.
INFO:evotuning:Starting epoch 9
DEBUG:root:Calculating average loss.


INFO:evotuning:Epoch 8: Estimated average training-set loss: 0.29967546463012695. Weights dumped.
INFO:evotuning:Starting epoch 10
DEBUG:root:Calculating average loss.


INFO:evotuning:Epoch 9: Estimated average training-set loss: 0.27984508872032166. Weights dumped.


In [62]:
params[0]["b"]

DeviceArray([2.896047 , 2.7695258, 2.985471 , ..., 3.035842 , 3.0082836,
             2.9999993], dtype=float32)

In [63]:
load_params_1900()["b"]

array([2.897026 , 2.7702985, 2.9864933, ..., 3.0359561, 3.0091407,
       2.999047 ], dtype=float32)

Here's what the problematic behaviour is. `h_avg` should take on identical values at every re-run of `get_reps`.

Instead, the value oscillates between two states. On the odd loops, they are one value, and on the even loops, they are another. See this in action below.

In [66]:
from jax_unirep import get_reps
import matplotlib.pyplot as plt

# params = load_params(folderpath="20200707/iter_1/")[0]
mut_seq = ['PROTEIN', 'PROTEI', 'PROTEIN']

logger.debug("")
logger.debug(":::::::NO CUSTOM PARAMS:::::::")
mut_rep = get_reps(mut_seq)[0]
# mut_rep_rerun = get_reps(mut_seq)[0]

for i in range(6):
    logger.debug("")
    logger.debug(f":::::::CALL {i} WITH CUSTOM:::::::")
    h_avg = get_reps(mut_seq, params=params[0])[0]


DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::NO CUSTOM PARAMS:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 4434875784
DEBUG:featurize.py:params is None, loading default params.
DEBUG:featurize.py:ID of params after checking None: 5162782800


DEBUG:featurize.py:rep_same_lengths:ID of params: 5162782800
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5162782800
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5162782800
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1098.8822021484375
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1088.96533203125
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 0 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1147.36279296875
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1100.1588134765625
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 1 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1133.589599609375
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1099.611328125
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 2 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1147.36279296875
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1100.1588134765625
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 3 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1133.589599609375
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1099.611328125
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 4 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1147.3629150390625
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1100.1588134765625
DEBUG:NOTEBOOK:
DEBUG:NOTEBOOK::::::::CALL 5 WITH CUSTOM:::::::
DEBUG:featurize.py:ID of params passed into get_reps: 5203122928
DEBUG:featurize.py:params were loaded from user.
DEBUG:featurize.py:ID of params after checking None: 5203122928


DEBUG:featurize.py:rep_same_lengths:ID of params: 5203122928
DEBUG:featurize.py:rep_same_lengths:sum of embedded_seqs: 46.749168395996094
DEBUG:layers.py:apply_fun:id of params: 5203122928
DEBUG:layers.py:apply_fun:sum of inputs: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch ID of params: 5203122928
DEBUG:layers.py:mlstm1900_batch sum of batch: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:layers.py:mlstm1900_batch sum of h_final: Traced<ShapedArray(float32[])>with<BatchTrace(level=0/0)>
DEBUG:featurize.py:rep_same_lengths:sum of h_final: 1133.589599609375
DEBUG:featurize.py:rep_same_lengths:sum of h_avg: 1099.611328125


This _oscillatory_ behaviour indicates to me that there's some cache-ing that might be going wrong.

I've spent a few hours trying to narrow down the bug. The main hypothesis I've tried is to check that the ID of the params being loaded are correct, as I wasn't sure whether they were being willy-nilly loaded elsewhere, but this turned out to be a dead-end: the IDs of the params passed in stays consistent over time, which means that the same params object is being used over loop runs. 

At this point, the parts that I don't know how to introspect into are the jax parts, i.e. vmap and lax.scan.

This is where I might have to ping for help with the jax devs.

Some source code surfaced up here for convenience that might help:

In [73]:
from jax_unirep.layers import mLSTM1900_batch, mLSTM1900
from jax_unirep.featurize import rep_same_lengths

mLSTM1900_batch??

Signature:
mLSTM1900_batch(
    params: Dict[str, jax.numpy.lax_numpy.ndarray],
    batch: jax.numpy.lax_numpy.ndarray,
) -> Tuple[jax.numpy.lax_numpy.ndarray, jax.numpy.lax_numpy.ndarray, jax.numpy.lax_numpy.ndarray]
Source:   
def mLSTM1900_batch(
    params: Dict[str, np.ndarray], batch: np.ndarray
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    LSTM layer implemented according to UniRep,
    found here:
    https://github.com/churchlab/UniRep/blob/master/unirep.py#L43,
    for a batch of data.

    This function processes a single embedded sequence,
    passed in as a two dimensional array,
    with number of rows being number of sequence positions,
    and the number of columns being the embedding of each sequence letter.

    :param params: All weights and biases for a single
        mLSTM1900 RNN cell.
    :param batch: One sequence embedded in a (n, 10) matrix,
        where `n` is the number of sequences
    :returns:
    """
    h_t = np.zeros(params["wmh"].shape[

**Does `lax.scan` do some caching?**

In [74]:
mLSTM1900??

Signature:
mLSTM1900(
    output_dim=1900,
    W_init=<function variance_scaling.<locals>.init at 0x1281b6560>,
    b_init=<function normal.<locals>.init at 0x1281b6ef0>,
)
Source:   
def mLSTM1900(output_dim=1900, W_init=glorot_normal(), b_init=normal()):
    """
    mLSTM cell from the UniRep paper, stax compatible

    This function works on a per-sequence basis,
    meaning that mapping over batches of sequences
    needs to happen outside this function, like this:

    .. code-block:: python

        def apply_fun_vmapped(x):
            return apply_fun(params=params, inputs=x)
        h_final, c_final, outputs = vmap(apply_fun_vmapped)(emb_seqs)

    It returns the average hidden, final hidden and final cell states
    of the mlstm.

    """

    def init_fun(rng, input_shape):
        """
        Initialize parameters for mLSTM1900

        output_dim:
            mlstm cell size -> (1900,)
        input_shape:
            one embedded sequence -> (n_letters, 10)
        output

In [75]:
rep_same_lengths??

Signature: rep_same_lengths(seqs: Iterable[str], params: Dict) -> Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]
Source:   
def rep_same_lengths(
    seqs: Iterable[str], params: Dict
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    This function generates representations of protein sequences that have the same length,
    by passing them through the UniRep mLSTM.

    :param seqs: A list of same length sequences as strings.
        If passing only a single sequence, it also needs to be passed inside a list.
    :returns: A tuple of np.arrays containing the reps.
        Each `np.array` has shape (n_sequences, 1900).
    """
    _, apply_fun = mLSTM1900()
    embedded_seqs = get_embeddings(seqs)
    logger.debug(f"rep_same_lengths:ID of params: {id(params)}")
    logger.debug(f"rep_same_lengths:sum of embedded_seqs: {np.sum(embedded_seqs)}")
    apply_fun = partial(apply_fun, params)
    h_final, c_final, h = vmap(apply_fun)(embedded_seqs)
    logger.debug(f"rep_same_len

**Could it be that `vmap` does some caching?**

In [76]:
get_reps??

Signature:
get_reps(
    seqs: Union[str, Iterable[str]],
    params: Union[Dict, NoneType] = None,
) -> Tuple[numpy.ndarray, numpy.ndarray, numpy.ndarray]
Source:   
def get_reps(
    seqs: Union[str, Iterable[str]], params: Optional[Dict] = None
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    This function generates representations of protein sequences using the
    1900 hidden-unit mLSTM model with pre-trained weights from the UniRep
    paper (https://github.com/churchlab/UniRep).

    Each element of the output 3-tuple is a `np.array`
    of shape (n_input_sequences, 1900):
    - `h_avg`: Average hidden state of the mLSTM over the whole sequence.
    - `h_final`: Final hidden state of the mLSTM
    - `c_final`: Final cell state of the mLSTM

    You should not use this function
    if you want to do further JAX-based computations
    on the output vectors!
    In that case, the `DeviceArray` futures returned by `mLSTM1900`
    should be passed directly into the next st